In [ ]:
import ast
import astor
class InputVisitor(ast.NodeVisitor):
    def __init__(self):
        self.user_inputs = {}

    def visit_Assign(self, node):
        for child in ast.walk(node.value):
            if isinstance(child, ast.Call):
                if isinstance(child.func, ast.Name):
                    if child.func.id == 'input':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                print(target.id)
                                if(str(target.id)  not in self.user_inputs):
                                   self.user_inputs[target.id]=str
                    elif child.func.id == 'int':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                self.user_inputs[target.id] = int
                                break
                    elif child.func.id == 'float':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                self.user_inputs[target.id] = float
                                break
        self.generic_visit(node)


def detect_user_inputs(code_file):
    with open(code_file, 'r') as f:
        code = f.read()
    tree = ast.parse(code)
    visitor = InputVisitor()
    visitor.visit(tree)
    return visitor.user_inputs


In [ ]:
import ast
import astor

class InputVisitor(ast.NodeTransformer):
    def __init__(self):
        self.user_inputs = {}
        self.param_names = ["param" + str(i) for i in range(1, 11)] # Değişken isimleri

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == "input":
            # input fonksiyonunu değiştir
            new_name = self.param_names.pop(0)
            self.user_inputs[new_name] = str
            new_node=ast.Name(id=new_name, ctx=ast.Load())
            return ast.copy_location(new_node, node)
        else:
            return self.generic_visit(node)

    


def detect_user_inputs(code_file):
    with open(code_file, 'r') as f:
        code = f.read()
    tree = ast.parse(code)
    visitor = InputVisitor()
    visitor.visit(tree)
    ast.fix_missing_locations(tree)
    new_code = astor.to_source(tree)
    with open('final_file1.py', 'w') as f:
        f.write(new_code)
    return visitor.user_inputs


In [ ]:
detect_user_inputs("code_2.py")